### Import Library

In [113]:
# import library
import pandas as pd
import numpy as np
import statsmodels
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_pinball_loss
import lightgbm as lgb
from IPython.display import clear_output

### Setting

In [11]:
# pandas setting
pd.options.display.max_rows = None
pd.options.display.max_columns = None

### Read Data

In [129]:
# read data
train_data = pd.read_csv("../data/train_data.csv", index_col='id')
test_data = pd.read_csv('../data/test_data.csv', index_col='id')
submissoin_data = pd.read_csv('../data/submission.csv')

In [36]:
# visualize train data
train_data.head() 

,date,highest,lowest,rain,ice1,ice2,ice3,oden1,oden2,oden3,oden4,hot1,hot2,hot3,dessert1,dessert2,dessert3,dessert4,dessert5,drink1,drink2,drink3,drink4,drink5,drink6,alcol1,alcol2,alcol3,snack1,snack2,snack3,bento1,bento2,bento3,bento4,tild1,tild2,men1,men2,men3,men4,men5,men6
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,4/11,21.9,12.4,0.0,25,72,26,10,23,52,35,180,254,270,42,58,50,59,67,54,45,28,49,22,8,63,51,59,26,21,35,56,46,70,27,12,12,57,30,41,38,37,35
2,4/12,25.9,13.9,0.0,30,85,33,9,18,42,26,202,219,235,22,36,5,28,37,69,54,35,58,22,9,77,66,72,36,32,63,8,14,23,9,5,8,19,9,13,26,4,16
3,4/13,20.9,11.9,0.0,21,68,28,12,22,57,31,164,210,223,20,41,5,30,32,46,38,24,45,26,9,81,69,74,36,25,57,9,12,19,6,4,9,23,9,11,33,4,13
4,4/14,18.8,11.4,0.0,19,62,35,13,29,62,33,193,242,251,21,43,5,28,32,35,31,22,37,23,8,88,92,81,20,10,45,54,43,74,28,15,17,55,35,46,46,51,46
5,4/15,22.1,13.5,19.0,21,72,32,10,24,44,33,218,271,274,44,63,47,59,63,51,39,20,52,28,14,116,135,119,31,21,39,52,36,69,26,4,9,54,33,39,40,41,40


In [75]:
train_data.shape

(350, 43)

In [23]:
# visualize test data
test_data.tail() 

,date,highest,lowest,rain
id,,,,
17,4/12,12.6,7.5,0.0
18,4/13,18.2,5.5,0.0
19,4/14,18.3,9.7,0.0
20,4/15,22.7,11.6,5.0
21,4/16,21.0,10.1,0.0


### Preprocess

In [96]:
columns = train_data.columns # all columns
train_columns = columns[1:4] # train data columns ['highest', 'lowest', 'rain']
target_columns = columns[4:] # target data columns

X_all = train_data[train_columns] # train data
Y_all = train_data[target_columns] # target data

X_train, X_valid, Y_train, Y_valid = train_test_split(X_all, Y_all, test_size=0.2, random_state=0)

### Train

In [118]:
# 分位点, itemごとにモデルを作成, 学習

quantiles = [0.01, 0.1, 0.5, 0.9, 0.99] # q
lgb_scores = [] # スコア
models = {} # model
verbose_eval = 0  # この数字を1にすると学習時のスコア推移がコマンドライン表示される
oof = np.zeros((len(X_valid), len(quantiles)*len(target_columns))) # out of fold


# item
for j, item in enumerate(target_columns):
    # 分位点
    for i, q in enumerate(quantiles): 
    
        # target データ
        y_train, y_valid = Y_train[item], Y_valid[item] 
        
        # 使用するパラメータ
        params = {
            'objective': 'quantile',  # 最小化させるべき損失関数
            'alpha': q, # quantiles
            'random_state': 0,  # 乱数シード
            'n_estimators': 10000, # 最大学習サイクル数。early_stopping使用時は大きな値を入力
            # 'max_depth': 5,
            # 'colsample_bytree': 0.5,
        }
        
        # モデル作成
        lgbr = lgb.LGBMRegressor(**params)
        
        # 学習
        lgbr.fit(
            X_train, 
            y_train, 
            eval_set=(X_valid, y_valid), 
            callbacks=[
                lgb.early_stopping(stopping_rounds=10, 
                verbose=True), # early_stopping用コールバック関数
                lgb.log_evaluation(verbose_eval) # コマンドライン出力用コールバック関数
            ]
        )
        
        # 学習させた model を保存
        models[f'{item}_{q}'] = lgbr
        
        # validationのベストスコアを保存
        lgb_scores.append(lgbr.best_score_['valid_0']['quantile'])
        
        # Out of Fold
        oof[:,int(i+len(quantiles)*j)] = lgbr.predict(X_valid)

# print(lgb_scores)

Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[368]	valid_0's quantile: 0.408663
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[70]	valid_0's quantile: 2.02026
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[59]	valid_0's quantile: 7.61152
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[103]	valid_0's quantile: 8.04935
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[59]	valid_0's quantile: 7.7654
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[392]	valid_0's quantile: 0.219813
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[148]	valid_0's quantile: 0.4562
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration i

In [120]:
# validの予測結果
oof_df = pd.DataFrame(Y_valid).reset_index(drop=True)
for j, item in enumerate(target_columns):
    for i, q in enumerate(quantiles):
        oof_df[f'{item}_{q}'] = oof[:,int(i+len(quantiles)*j)]
oof_df = oof_df.sort_index(axis='columns')
clear_output()
# oof_df.sample(n=10)
oof_df.head(10)

,alcol1,alcol1_0.01,alcol1_0.1,alcol1_0.5,alcol1_0.9,alcol1_0.99,alcol2,alcol2_0.01,alcol2_0.1,alcol2_0.5,alcol2_0.9,alcol2_0.99,alcol3,alcol3_0.01,alcol3_0.1,alcol3_0.5,alcol3_0.9,alcol3_0.99,bento1,bento1_0.01,bento1_0.1,bento1_0.5,bento1_0.9,bento1_0.99,bento2,bento2_0.01,bento2_0.1,bento2_0.5,bento2_0.9,bento2_0.99,bento3,bento3_0.01,bento3_0.1,bento3_0.5,bento3_0.9,bento3_0.99,bento4,bento4_0.01,bento4_0.1,bento4_0.5,bento4_0.9,bento4_0.99,dessert1,dessert1_0.01,dessert1_0.1,dessert1_0.5,dessert1_0.9,dessert1_0.99,dessert2,dessert2_0.01,dessert2_0.1,dessert2_0.5,dessert2_0.9,dessert2_0.99,dessert3,dessert3_0.01,dessert3_0.1,dessert3_0.5,dessert3_0.9,dessert3_0.99,dessert4,dessert4_0.01,dessert4_0.1,dessert4_0.5,dessert4_0.9,dessert4_0.99,dessert5,dessert5_0.01,dessert5_0.1,dessert5_0.5,dessert5_0.9,dessert5_0.99,drink1,drink1_0.01,drink1_0.1,drink1_0.5,drink1_0.9,drink1_0.99,drink2,drink2_0.01,drink2_0.1,drink2_0.5,drink2_0.9,drink2_0.99,drink3,drink3_0.01,drink3_0.1,drink3_0.5,drink3_0.9,drink3_0.99,drink4,drink4_0.01,drink4_0.1,drink4_0.5,drink4_0.9,drink4_0.99,drink5,drink5_0.01,drink5_0.1,drink5_0.5,drink5_0.9,drink5_0.99,drink6,drink6_0.01,drink6_0.1,drink6_0.5,drink6_0.9,drink6_0.99,hot1,hot1_0.01,hot1_0.1,hot1_0.5,hot1_0.9,hot1_0.99,hot2,hot2_0.01,hot2_0.1,hot2_0.5,hot2_0.9,hot2_0.99,hot3,hot3_0.01,hot3_0.1,hot3_0.5,hot3_0.9,hot3_0.99,ice1,ice1_0.01,ice1_0.1,ice1_0.5,ice1_0.9,ice1_0.99,ice2,ice2_0.01,ice2_0.1,ice2_0.5,ice2_0.9,ice2_0.99,ice3,ice3_0.01,ice3_0.1,ice3_0.5,ice3_0.9,ice3_0.99,men1,men1_0.01,men1_0.1,men1_0.5,men1_0.9,men1_0.99,men2,men2_0.01,men2_0.1,men2_0.5,men2_0.9,men2_0.99,men3,men3_0.01,men3_0.1,men3_0.5,men3_0.9,men3_0.99,men4,men4_0.01,men4_0.1,men4_0.5,men4_0.9,men4_0.99,men5,men5_0.01,men5_0.1,men5_0.5,men5_0.9,men5_0.99,men6,men6_0.01,men6_0.1,men6_0.5,men6_0.9,men6_0.99,oden1,oden1_0.01,oden1_0.1,oden1_0.5,oden1_0.9,oden1_0.99,oden2,oden2_0.01,oden2_0.1,oden2_0.5,oden2_0.9,oden2_0.99,oden3,oden3_0.01,oden3_0.1,oden3_0.5,oden3_0.9,oden3_0.99,oden4,oden4_0.01,oden4_0.1,oden4_0.5,oden4_0.9,oden4_0.99,snack1,snack1_0.01,snack1_0.1,snack1_0.5,snack1_0.9,snack1_0.99,snack2,snack2_0.01,snack2_0.1,snack2_0.5,snack2_0.9,snack2_0.99,snack3,snack3_0.01,snack3_0.1,snack3_0.5,snack3_0.9,snack3_0.99,tild1,tild1_0.01,tild1_0.1,tild1_0.5,tild1_0.9,tild1_0.99,tild2,tild2_0.01,tild2_0.1,tild2_0.5,tild2_0.9,tild2_0.99
0,47,37.468559,47.851487,73.35,99.449993,108.635364,39,18.100,30.452801,64.689500,92.132681,114.0,44,32.819989,39.924521,70.988951,99.700000,109.255484,49,8.000,9.779172,46.40,53.9,58.0,42,10.000000,12.21,39.6750,47.20,51.001479,72,14.612580,18.41,66.2460,75.5610,80.169459,24,6.745813,7.8100,25.80,30.065995,32.080004,33,19.00000,20.194299,26.80,45.730000,55.888837,54,36.110,38.00000,45.05,65.92,81.540005,12,5.0,5.0,6.0,50.269878,61.340005,37,26.000000,26.37190,30.00,60.039993,67.951011,41,32.039,33.937118,38.0000,66.878997,88.392007,16,13.251122,17.288633,17.761950,20.000053,78.00000,18,18.019427,18.567787,19.332547,39.093040,67.916366,12,10.759179,8.215509,12.321909,15.953569,63.730019,27,27.000000,27.736201,29.983170,33.636709,79.002855,23,22.100,23.043406,25.791729,27.700842,30.0,11,7.100,10.164847,10.698437,15.212720,16.0,197,83.768187,159.977036,193.434581,194.789001,230.000000,251,197.980754,214.472835,239.144971,255.433489,267.467047,248,197.348093,215.555194,253.250628,278.939622,283.417537,17,14.210,17.191980,18.720039,20.533994,85.982266,50,37.000000,48.879056,50.839602,51.000483,109.013705,27,24.943402,26.932618,30.420269,33.542933,56.853822,47,17.183199,19.28,46.50,56.90,71.0,32,8.179756,9.1900,28.30,34.905262,37.027432,37,10.686189,12.2,33.30,42.394714,53.280004,43,24.719989,28.469571,36.560,45.050747,59.0,41,3.0,4.0,36.756750,43.936814,53.840005,35,13.000000,15.13,34.7320,42.311002,63.82001,13,0.200001,11.987483,17.224997,16.569914,96.870412,26,0.340001,23.027377,26.910349,28.111783,125.060015,58,0.660003,53.495028,59.109127,87.200023,122.009375,40,0.520002,33.287663,45.284592,43.07

### Submission

In [130]:
X_test = test_data[['highest', 'lowest', 'rain']]

for i, item in enumerate(target_columns):
    for j, q in enumerate(quantiles):
        submissoin_data[f'{item}_{q}'] = models[f'{item}_{q}'].predict(X_test)

submissoin_data

,id,ice1_0.01,ice1_0.1,ice1_0.5,ice1_0.9,ice1_0.99,ice2_0.01,ice2_0.1,ice2_0.5,ice2_0.9,ice2_0.99,ice3_0.01,ice3_0.1,ice3_0.5,ice3_0.9,ice3_0.99,oden1_0.01,oden1_0.1,oden1_0.5,oden1_0.9,oden1_0.99,oden2_0.01,oden2_0.1,oden2_0.5,oden2_0.9,oden2_0.99,oden3_0.01,oden3_0.1,oden3_0.5,oden3_0.9,oden3_0.99,oden4_0.01,oden4_0.1,oden4_0.5,oden4_0.9,oden4_0.99,hot1_0.01,hot1_0.1,hot1_0.5,hot1_0.9,hot1_0.99,hot2_0.01,hot2_0.1,hot2_0.5,hot2_0.9,hot2_0.99,hot3_0.01,hot3_0.1,hot3_0.5,hot3_0.9,hot3_0.99,dessert1_0.01,dessert1_0.1,dessert1_0.5,dessert1_0.9,dessert1_0.99,dessert2_0.01,dessert2_0.1,dessert2_0.5,dessert2_0.9,dessert2_0.99,dessert3_0.01,dessert3_0.1,dessert3_0.5,dessert3_0.9,dessert3_0.99,dessert4_0.01,dessert4_0.1,dessert4_0.5,dessert4_0.9,dessert4_0.99,dessert5_0.01,dessert5_0.1,dessert5_0.5,dessert5_0.9,dessert5_0.99,drink1_0.01,drink1_0.1,drink1_0.5,drink1_0.9,drink1_0.99,drink2_0.01,drink2_0.1,drink2_0.5,drink2_0.9,drink2_0.99,drink3_0.01,drink3_0.1,drink3_0.5,drink3_0.9,drink3_0.99,drink4_0.01,drink4_0.1,drink4_0.5,drink4_0.9,drink4_0.99,drink5_0.01,drink5_0.1,drink5_0.5,drink5_0.9,drink5_0.99,drink6_0.01,drink6_0.1,drink6_0.5,drink6_0.9,drink6_0.99,alcol1_0.01,alcol1_0.1,alcol1_0.5,alcol1_0.9,alcol1_0.99,alcol2_0.01,alcol2_0.1,alcol2_0.5,alcol2_0.9,alcol2_0.99,alcol3_0.01,alcol3_0.1,alcol3_0.5,alcol3_0.9,alcol3_0.99,snack1_0.01,snack1_0.1,snack1_0.5,snack1_0.9,snack1_0.99,snack2_0.01,snack2_0.1,snack2_0.5,snack2_0.9,snack2_0.99,snack3_0.01,snack3_0.1,snack3_0.5,snack3_0.9,snack3_0.99,bento1_0.01,bento1_0.1,bento1_0.5,bento1_0.9,bento1_0.99,bento2_0.01,bento2_0.1,bento2_0.5,bento2_0.9,bento2_0.99,bento3_0.01,bento3_0.1,bento3_0.5,bento3_0.9,bento3_0.99,bento4_0.01,bento4_0.1,bento4_0.5,bento4_0.9,bento4_0.99,tild1_0.01,tild1_0.1,tild1_0.5,tild1_0.9,tild1_0.99,tild2_0.01,tild2_0.1,tild2_0.5,tild2_0.9,tild2_0.99,men1_0.01,men1_0.1,men1_0.5,men1_0.9,men1_0.99,men2_0.01,men2_0.1,men2_0.5,men2_0.9,men2_0.99,men3_0.01,men3_0.1,men3_0.5,men3_0.9,men3_0.99,men4_0.01,men4_0.1,men4_0.5,men4_0.9,men4_0.99,men5_0.01,men5_0.1,men5_0.5,men5_0.9,men5_0.99,men6_0.01,men6_0.1,men6_0.5,men6_0.9,men6_0.99
0,1,14.210000,19.116386,20.539997,22.304491,85.982266,37.000000,63.054476,64.649809,64.922085,109.013705,25.633735,28.046353,31.393997,34.549681,56.853822,1.200000,27.152439,41.461456,47.992548,96.870412,2.310000,45.737223,42.141268,61.527511,125.060015,5.420000,71.924579,107.488319,104.816701,156.222474,3.200000,56.688860,69.865719,68.433701,69.884507,83.768187,133.683465,139.906523,155.455168,230.0,197.064219,201.562122,218.913565,239.523457,253.528853,197.348093,207.584817,222.581420,244.321380,283.417537,19.11178,20.965964,27.80,46.30,55.888837,36.110,39.0317,46.25,66.649999,81.540005,5.0,5.0,6.70,50.114383,61.340005,26.000000,26.29000,32.40,60.409993,67.951011,32.000,33.283905,39.750,68.130240,88.392007,15.880382,36.085672,39.206510,41.997259,78.0,25.808736,30.014003,35.746899,41.278041,67.916366,10.759179,20.891491,21.888529,26.075642,63.730019,31.0,38.792562,41.834376,44.917511,79.002855,22.0,22.000000,24.860267,26.050063,30.0,7.100,7.957920,8.000000,9.001280,16.0,37.655983,40.418723,72.00,98.259993,108.635364,18.02,20.016617,65.988250,93.575649,114.000000,32.819989,36.830405,65.413045,98.039999,109.255484,16.031154,20.03,26.7290,35.70,42.481006,9.100,12.013234,22.109016,32.32,37.0,25.939935,33.500000,43.80,55.48,63.260005,8.0,8.071006,46.40,54.30,58.0,10.000000,12.10,36.3200,46.90,51.001479,14.612580,17.94,59.134,75.3080,80.169459,6.745813,8.0100,26.10,29.755705,32.380004,4.000,4.870201,5.00,15.02,20.0,6.1,7.10,9.20,14.029358,18.0,17.183199,19.00,46.5,57.00,67.0,8.997261,9.0840,26.9,34.313325,37.027432,10.686189,12.10,34.45,46.275299,53.019004,24.819989,28.106479,36.71,46.971075,59.0,3.100,4.0,31.91625,45.578589,53.479006,13.000000,14.90,30.89925,44.991661,61.65401
1,2,14.210000,17.836453,19.681642,21.820265,85.982266,37.000000,51.304055,56.303054,57.017883,109.013705,24.998822,26.729960,30.454738,33.743608,56.853822,1.200000,18.3619

In [131]:
# save to csv file
submissoin_data.to_csv('../work/submission.csv', index=False)